# CNN-based Brain Tumour Segmentation Network
## Import packages
Please make sure you have all the required packages installed. 

In [1]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import ipywidgets as widgets

from matplotlib.widgets import Slider
from ipywidgets import interact, interactive, fixed, interact_manual

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


2024-07-11 19:13:47.799089: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-11 19:13:47.880353: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-11 19:13:49.149434: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with 

## Visualise MRI Volume Slices and Segmentation Maps
Each MRI image contains information about a three-dimensional (3D) volume of space. An MRI image is composed of a number of voxels, which is like pixels in 2D images. Here try to visualise the axial plane (usually has a higher resolution) of some of the volumes and the corresponding segmentation maps.

In [2]:
# Data Visualization
# Choose set on the selection bar, then use the trackbar for moving up and down

# messages
no_selection_hint = "Nothing Selected"

# path related
img_path = 'dataset_segmentation/'
train_path = os.path.join(img_path, "train")


# global variables 
view_pla_path = None
view_seg_path = None
view_pla_load = None
view_seg_load = None
slices = None
sliders = None

def update_slice(val):
    global view_pla_load
    global view_seg_load
    imgfla = view_pla_load[:,:,val]
    imgseg = view_seg_load[:,:,val]
    
    plt.figure(figsize=(12, 6))
    plt.subplot(1,2,1)
    plt.imshow(imgfla, cmap='gray')
    plt.title('FLA')

    plt.subplot(1,2,2)
    plt.imshow(imgseg, cmap='gray')
    plt.title('SEG')
    plt.show()
    return()

def update_set(strval):
    global view_pla_path 
    global view_seg_path
    global view_pla_load
    global view_seg_load
    global slices
    global sliders
    try:
        if sliders is not None:
            sliders.close()
    except NameError:
        pass
    if strval['type'] == 'change' and strval['name'] == 'value':
        set_str = strval['new']
    # print(set_str)
    if set_str == no_selection_hint:
        return()
    view_pla_path = os.path.join(train_path, set_str, set_str + "_fla.nii.gz")
    view_seg_path = os.path.join(train_path, set_str, set_str + "_seg.nii.gz")

    view_pla_load = nib.load(view_pla_path).get_fdata()
    view_seg_load = nib.load(view_seg_path).get_fdata()

    slices = view_pla_load.shape
    sliders = interactive(update_slice, val=widgets.IntSlider(value=0, min=0, max=slices[2]-1, step=1) )
    display(sliders)
    return()

dataset_subfolder = []

for CLASS in os.listdir(train_path):
    if not CLASS.startswith('.'):
        dataset_subfolder.append(CLASS)

dataset_subfolder.sort()
dataset_subfolder.insert(0, no_selection_hint)

dropdown = widgets.Dropdown(options=dataset_subfolder, value=no_selection_hint, description='Data Select')

dropdown.observe(update_set, names='value')
display(dropdown)





Dropdown(description='Data Select', options=('Nothing Selected', '001', '002', '003', '004', '005', '006', '00…

interactive(children=(IntSlider(value=0, description='val', max=154), Output()), _dom_classes=('widget-interac…

interactive(children=(IntSlider(value=0, description='val', max=154), Output()), _dom_classes=('widget-interac…

## Data preprocessing (Optional)

Images in the original dataset are usually in different sizes, so sometimes we need to resize and normalise (z-score is commonly used in preprocessing the MRI images) them to fit the CNN model. Depending on the images you choose to use for training your model, some other preprocessing methods. If preprocessing methods like cropping is applied, remember to convert the segmentation result back to its original size. 

## Train-time data augmentation
Generalizability is crucial to a deep learning model and it refers to the performance difference of a model when evaluated on the seen data (training data) versus the unseen data (testing data). Improving the generalizability of these models has always been a difficult challenge. 

**Data Augmentation** is an effective way of improving the generalizability, because the augmented data will represent a more comprehensive set of possible data samples and minimizing the distance between the training and validation/testing sets.

There are many data augmentation methods you can choose in this projects including rotation, shifting, flipping, etc.

You are encouraged to try different augmentation method to get the best segmentation result.


## Get the data generator ready

In [4]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=32, dim=(240,240), n_channels=3,
                 n_classes=2, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y 

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            # Add data augmentation here
            X[i,] = np.load(ID)

            # Store class
            y[i] = min(1,np.sum(np.load(ID.split('_')[0]+'_seg.npy')))

        return X, tf.keras.utils.to_categorical(y, num_classes=self.n_classes)

## Define a metric for the performance of the model
Dice score is used here to evaluate the performance of your model.
More details about the Dice score and other metrics can be found at 
https://towardsdatascience.com/metrics-to-evaluate-your-semantic-segmentation-model-6bcb99639aa2. Dice score can be also used as the loss function for training your model.

## Build your own model here
The U-Net (https://link.springer.com/chapter/10.1007/978-3-319-24574-4_28) structure is widely used for the medical image segmentation task. You can build your own model or modify the UNet by changing the hyperparameters for our task. If you choose to use Keras, more information about the Keras layers including Conv2D, MaxPooling and Dropout can be found at https://keras.io/api/layers/.

In [18]:
import tensorflow as tf
import numpy as np
import cv2
import glob
import itertools
 
class UNet:
    def __init__(
            self,
            input_width,
            input_height,
            num_classes,
            train_images,
            train_instances,
            val_images,
            val_instances,
            epochs,
            lr,
            lr_decay,
            batch_size,
            save_path
    ):
        self.input_width = input_width
        self.input_height = input_height
        self.num_classes = num_classes
        self.train_images = train_images
        self.train_instances = train_instances
        self.val_images = val_images
        self.val_instances = val_instances
        self.epochs = epochs
        self.lr = lr
        self.lr_decay = lr_decay
        self.batch_size = batch_size
        self.save_path = save_path
 
    def leftNetwork(self, inputs):
        x = tf.keras.layers.Conv2D(64, (3, 3), padding='valid', activation='relu')(inputs)
        o_1 = tf.keras.layers.Conv2D(64, (3, 3), padding='valid', activation='relu')(x)
        x = tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2, 2))(o_1)
 
        x = tf.keras.layers.Conv2D(128, (3, 3), padding='valid', activation='relu')(x)
        o_2 = tf.keras.layers.Conv2D(128, (3, 3), padding='valid', activation='relu')(x)
        x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(o_2)
 
        x = tf.keras.layers.Conv2D(256, (3, 3), padding='valid', activation='relu')(x)
        o_3 = tf.keras.layers.Conv2D(256, (3, 3), padding='valid', activation='relu')(x)
        x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(o_3)
 
        x = tf.keras.layers.Conv2D(512, (3, 3), padding='valid', activation='relu')(x)
        o_4 = tf.keras.layers.Conv2D(512, (3, 3), padding='valid', activation='relu')(x)
        x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(o_4)
 
        x = tf.keras.layers.Conv2D(1024, (3, 3), padding='valid', activation='relu')(x)
        o_5 = tf.keras.layers.Conv2D(1024, (3, 3), padding='valid', activation='relu')(x)
 
        return [o_1, o_2, o_3, o_4, o_5]
 
    def rightNetwork(self, inputs):
        c_1, c_2, c_3, c_4, o_5 = inputs
 
        o_5 = tf.keras.layers.UpSampling2D((2, 2))(o_5)
        x = tf.keras.layers.concatenate([tf.keras.layers.Cropping2D(4)(c_4), o_5], axis=3)
        x = tf.keras.layers.Conv2D(512, (3, 3), padding='valid', activation='relu')(x)
        x = tf.keras.layers.Conv2D(512, (3, 3), padding='valid', activation='relu')(x)
        x = tf.keras.layers.UpSampling2D((2, 2))(x)
 
        x = tf.keras.layers.concatenate([tf.keras.layers.Cropping2D(16)(c_3), x], axis=3)
        x = tf.keras.layers.Conv2D(256, (3, 3), padding='valid', activation='relu')(x)
        x = tf.keras.layers.Conv2D(256, (3, 3), padding='valid', activation='relu')(x)
        x = tf.keras.layers.UpSampling2D((2, 2))(x)
 
        x = tf.keras.layers.concatenate([tf.keras.layers.Cropping2D(40)(c_2), x], axis=3)
        x = tf.keras.layers.Conv2D(128, (3, 3), padding='valid', activation='relu')(x)
        x = tf.keras.layers.Conv2D(128, (3, 3), padding='valid', activation='relu')(x)
        x = tf.keras.layers.UpSampling2D((2, 2))(x)
 
        x = tf.keras.layers.concatenate([tf.keras.layers.Cropping2D(88)(c_1), x], axis=3)
        x = tf.keras.layers.Conv2D(64, (3, 3), padding='valid', activation='relu')(x)
        x = tf.keras.layers.Conv2D(64, (3, 3), padding='valid', activation='relu')(x)
        x = tf.keras.layers.Conv2D(self.num_classes, (1, 1), padding='valid')(x)
        x = tf.keras.layers.Activation('softmax')(x)
 
        return x
 
    def build_model(self):
        inputs = tf.keras.Input(shape=[self.input_height, self.input_width, 3])
        left_output = self.leftNetwork(inputs)
        right_output = self.rightNetwork(left_output)

        model = tf.keras.Model(inputs=inputs, outputs=right_output)
 
        return model
 
    def train(self):
        G_train = self.dataGenerator(model='training')
        G_eval = self.dataGenerator(model='validation')
 
        model = self.build_model()
        # model = tf.keras.models.load_model('trained_model.h5')
        print("Built")
        model.compile(
            optimizer=tf.keras.optimizers.Adam(self.lr, self.lr_decay),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        model.fit_generator(
            G_train, 5, validation_data=G_eval, validation_steps=5, epochs=self.epochs
        )
        model.save(self.save_path)
 
    def dataGenerator(self, model):
        if model == 'training':
            images = glob.glob(self.train_images + '*.jpg')
            images.sort()
            instances = glob.glob(self.train_instances + '*.jpg')
            instances.sort()
            zipped = itertools.cycle(zip(images, instances))
            while True:
                x_train = []
                y_train = []
                for _ in range(self.batch_size):
                    img, seg = next(zipped)
                    img = cv2.resize(cv2.imread(img, 1), (self.input_width, self.input_height)) / 255.0
                    seg = tf.keras.utils.to_categorical(cv2.imread(seg, 0), self.num_classes)
                    x_train.append(img)
                    y_train.append(seg)
 
                yield np.array(x_train), np.array(y_train)
 
        if model == 'validation':
            images = glob.glob(self.val_images + '*.jpg')
            images.sort()
            instances = glob.glob(self.val_instances + '*.jpg')
            instances.sort()
            zipped = itertools.cycle(zip(images, instances))
 
            while True:
                x_eval = []
                y_eval = []
                for _ in range(self.batch_size):
                    img, seg = next(zipped)
                    img = cv2.resize(cv2.imread(img, 1), (self.input_width, self.input_height)) / 255.0
                    seg = tf.keras.utils.to_categorical(cv2.imread(seg, 0), self.num_classes)
                    x_eval.append(img)
                    y_eval.append(seg)
 
                yield np.array(x_eval), np.array(y_eval)

## Train your model here
Once you defined the model and data generator, you can start training your model.

In [19]:
unet = UNet(
    input_width=572,
    input_height=572,
    num_classes=3,
    train_images='./Train/image/',
    train_instances='./Train/masks/',
    val_images='./Val/image/',
    val_instances='./Val/masks/',
    epochs=100,
    lr=0.0001,
    lr_decay=0.00001,
    batch_size=8,
    save_path='model.h5'
)

unet.train()

Built
Epoch 1/100


/tmp/ipykernel_15371/3672649565.py:106: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(
2024-07-11 15:07:49.137525: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at conv_ops_fused_impl.h:656 : UNKNOWN: CUDNN failed to allocate the scratch space for the runner or to find a working no-scratch runner.


UnknownError: Graph execution error:

Detected at node 'model_4/conv2d_120/Relu' defined at (most recent call last):
    File "/home/terence/anaconda3/envs/tf-gpu/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/terence/anaconda3/envs/tf-gpu/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/terence/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "/home/terence/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/traitlets/config/application.py", line 1075, in launch_instance
      app.start()
    File "/home/terence/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 739, in start
      self.io_loop.start()
    File "/home/terence/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 205, in start
      self.asyncio_loop.run_fore

## Save the model
Once your model is trained, remember to save it for testing.

## Run the model on the test set
After your last Q&A session, you will be given the test set. Run your model on the test set to get the segmentation results and submit your results in a .zip file. If the MRI image is named '100_fla.nii.gz', save your segmentation result as '100_seg.nii.gz'. 